In [1]:
%run read_file.ipynb


Matplotlib is building the font cache; this may take a moment.


:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-84047dd5-525b-43ae-b809-77e671d451b0;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (40ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (22ms)
:: resolution report :: resolve 667ms :: artifacts dl 63ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	--------

23/03/24 01:38:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/24 01:38:56 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
%run build_adjacency.ipynb

In [8]:
df_epm = readCheckpoint(size = "medium")
df_epm = df_epm.limit(50000)
df_ohe_test = build_adjaceny(df_epm)

 1:25AM UTC on Mar 13, 2023 --- read and cache time: 0.3010730743408203 seconds ---
397
length actIDs: 125
elapsed time: 1.6015713214874268
8f18426d-d87e-479f-802d-d6f49a34a1e1


no birth event for 17 events.
elapsed time 2: 253.75643801689148


In [9]:
df_ohe_test.printSchema()

root
 |-- order: integer (nullable = false)
 |-- parent_objectid: string (nullable = true)
 |-- parent_timestamp: string (nullable = true)
 |-- parent_object: string (nullable = true)
 |-- parent_parent_image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- parent_USER: string (nullable = true)
 |-- child_objectid: string (nullable = true)
 |-- child_timestamp: string (nullable = true)
 |-- child_object: string (nullable = true)
 |-- child_parent_image_path: string (nullable = true)
 |-- child_image_path: string (nullable = true)
 |-- child_USER: string (nullable = true)



In [10]:
df_ohe_test.select(col("child_image_path") == "PING.EXE").count()

+--------------------+
|    child_image_path|
+--------------------+
|\Device\HarddiskV...|
|              System|
|\Device\HarddiskV...|
|\Device\HarddiskV...|
|\Device\HarddiskV...|
|\Device\HarddiskV...|
|\Device\HarddiskV...|
|\Device\HarddiskV...|
|\Device\HarddiskV...|
|\\?\C:\Program Fi...|
|\Device\HarddiskV...|
|\Device\HarddiskV...|
|            PING.EXE|
+--------------------+



In [11]:
df_ohe_test.select(col("child_image_path") == "PING.EXE").count()

431

In [21]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
#turn into numeric index before encoding
indexer = StringIndexer(inputCol='parent_object', outputCol='p_obj_numeric')
indexer_fitted = indexer.fit(df_ohe_test)
df_indexed = indexer_fitted.transform(df_ohe_test)

encoder = OneHotEncoder(inputCols=['p_obj_numeric'], outputCols=['p_obj_sparse'],dropLast=False)
df_onehot = encoder.fit(df_indexed).transform(df_indexed)
df_onehot.select('parent_object','p_obj_sparse').limit(20).show()

+-------------+-------------+
|parent_object| p_obj_sparse|
+-------------+-------------+
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
|      PROCESS|(1,[0],[1.0])|
+-------------+-------------+



In [18]:
df_onehot.printSchema()

root
 |-- order: integer (nullable = false)
 |-- parent_objectid: string (nullable = true)
 |-- parent_timestamp: string (nullable = true)
 |-- parent_object: string (nullable = true)
 |-- parent_parent_image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- parent_USER: string (nullable = true)
 |-- child_objectid: string (nullable = true)
 |-- child_timestamp: string (nullable = true)
 |-- child_object: string (nullable = true)
 |-- child_parent_image_path: string (nullable = true)
 |-- child_image_path: string (nullable = true)
 |-- child_USER: string (nullable = true)
 |-- p_obj_numeric: double (nullable = false)
 |-- p_obj_sparse: vector (nullable = true)



In [22]:
#create sparse vecotrs for child objects
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
#turn into numeric index before encoding
indexer = StringIndexer(inputCol='child_object', outputCol='c_obj_numeric')
indexer_fitted = indexer.fit(df_onehot)
df_indexed = indexer_fitted.transform(df_onehot)

encoder = OneHotEncoder(inputCols=['c_obj_numeric'], outputCols=['c_obj_sparse'],dropLast=False)
df_onehot = encoder.fit(df_indexed).transform(df_indexed)
df_onehot = df_onehot.drop('c_obj_numeric', 'p_obj_numeric')
df_onehot.select('p_obj_sparse','c_obj_sparse').limit(20).show()

+-------------+-------------+
| p_obj_sparse| c_obj_sparse|
+-------------+-------------+
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[1],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[1],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[1],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
|(1,[0],[1.0])|(2,[0],[1.0])|
+-------------+-------------+



In [23]:
#combine sparse vectors and turn into string
from pyspark.ml.feature import VectorAssembler
# Use VectorAssembler to concatenate the two sparse vectors
assembler = VectorAssembler(inputCols=['p_obj_sparse', 'c_obj_sparse'], outputCol="comb_sparse")
df_onehot_comb = assembler.transform(df_onehot)

In [24]:
df_onehot_comb.select('comb_sparse').limit(5).show()

+-------------+
|  comb_sparse|
+-------------+
|[1.0,1.0,0.0]|
|[1.0,1.0,0.0]|
|[1.0,0.0,1.0]|
|[1.0,1.0,0.0]|
|[1.0,1.0,0.0]|
+-------------+



In [25]:
#create a timestamp timediff, bucketize, and then create a one hot vector, and then combine all three
#question: does order matter

df_onehot_comb = df_onehot_comb.withColumn('from_timestamp',to_timestamp(col('parent_timestamp')))\
  .withColumn('end_timestamp', to_timestamp(col('child_timestamp')))\
  .withColumn('DiffInSeconds',col("end_timestamp").cast("long") - col('from_timestamp').cast("long")) 
                                           

In [ ]:
# Use QuantileDiscretizer to calculate the bin boundaries
quantile_discretizer = QuantileDiscretizer(numBuckets=5, inputCol="DiffInSeconds", outputCol="binned_diff")
df_onehot_comb = quantile_discretizer.fit(df_onehot_comb).transform(df_onehot_comb)

# Convert the binned column to a sparse vector -> may want to consider a different method for this.
sparse_vectors = df_onehot_comb.rdd.map(lambda x: (x[0], Vectors.sparse(5, {x[2]: 1.0})))
sparse_df = spark.createDataFrame(sparse_vectors, ["id", "sparse_vector"])

# Show the resulting sparse vectors
sparse_df.show()

In [ ]:
#function to accept an event trace data frame and then encode it. Ideally we run this twice, once on the malicious
#traces and again on benign traces. This implies that we run graphframes on both sets of events independently. 

def trace_encode(df, col_list):
    
    #step one accept the event trace transpose it, and explode it. 

    #add "trace index" to keep track of traces. important for transposing back
    df_transp = df.withColumn("Trace", (monotonically_increasing_id() + 1))
    df_transp = df_transp.select("Trace", 
                                 *[col for col in df_transp.columns if col != "Trace"])

    #drop all vertices
    df_transp = df_transp.drop('a','b','c','d','e','f','g')

    #transpose rows 
    stacked_df = df_transp.selectExpr(
        "Trace", 
        "posexplode(array(e1, e2, e3, e4, e5, e6)) as (pos, col)"
    ).select(
        "Trace", 
        expr('''CASE pos 
        WHEN 0 THEN 'e1' 
        WHEN 1 THEN 'e2'
        WHEN 2 THEN 'e3'
        WHEN 3 THEN 'e4'
        WHEN 4 THEN 'e5'
        ELSE 'e6' END''').alias("event"),
        "col"
    ).orderBy("Trace","event")

    #explode columns
    stacked_df = stacked_df.select(*stacked_df.columns, "col.*").drop('col')
    
    
    
    
    
    return df

In [3]:
#start work on concatenating columns and then string vector. 
#first read in dataframe and check to see if the duplicate events are gone (w/o mike for the last part). 

df = spark.read.parquet(f"{s3_url_trusted}/prod/graph/first_events")\
            .withColumn("image_path", getFileUDF(col("image_path"))) \
            .withColumn("parent_image_path", getFileUDF(col("parent_image_path"))) \
            .withColumn("new_path", getFileUDF(col("new_path"))) \
            .withColumn("file_path", getFileUDF(col("file_path"))) \
            .cache()

In [110]:
%run ./etl_trusted_features.ipynb
df = readFirstEvents()
#split into malicious and benign and count
df_mal = df.where(col('malicious') == 1)
print("malicious count: " + str(df_mal.count()))

10:33PM UTC on Mar 18, 2023 --- read time: 0.5996990203857422 seconds ---
malicious count: 3642


In [111]:
from graphframes import GraphFrame

#create trace matrix from malicious events for speed. 
# Create distinct vertices with source as actorid, destination as objectid for malicious
src_vertices = df_mal.selectExpr('actorID as id').distinct()
dst_vertices = df_mal.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_mal.selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                      'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d); (d)-[e4]->(e); (e)-[e5]->(f); (f)-[e6]->(g)")

#create paths and count
# filter paths to only those where all edges are connected
connected_paths = motifs6.filter("e1.dst = e2.src and e2.dst = e3.src and e3.dst = e4.src and e4.dst = e5.src and e5.dst = e6.src").cache()
print("event traces: "+str(connected_paths.count()))

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


event traces: 21135


In [36]:
#run connected_paths through trace and encode.
%run trace_encode.ipynb

In [37]:
list_cols = ['object','action',
             'image_path','parent_image_path']
df_test, dict_test = trace_encode(connected_paths,list_cols)

['event', 'src', 'dst', 'hostname', 'user_name', 'privileges', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user']


elapsed time: 5.628903388977051


In [38]:
df_test.limit(15).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,timestamp,malicious,object_sparse,action_sparse,image_path_sparse,parent_image_path_sparse
0,1,2019-09-23 18:44:59.171,1,"(1.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0)"
1,1,2019-09-23 18:44:59.307,1,"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0)"
2,1,2019-09-23 18:44:59.307,1,"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0)"
3,1,2019-09-23 18:44:59.307,1,"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0)"
4,1,2019-09-23 18:44:59.307,1,"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0)"
5,1,2019-09-23 18:44:59.515,1,"(0.0, 1.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 1.0)"
6,2,2019-09-23 18:44:59.307,1,"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0)"
7,2,2019-09-23 18:44:59.307,1,"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0)"
8,2,2019-09-23 18:44:59.307,1,"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0)"
9,2,2019-09-23 18:44:59.307,1,"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0)"


In [41]:
#now let's figure out how to string concatenate and then index
#I think we're going to want to assemble the vectors, turn into a string, then string index
#first vector assemble

#combine sparse vectors and turn into string
from pyspark.ml.feature import VectorAssembler
# Use VectorAssembler to concatenate the two sparse vectors
assembler = VectorAssembler(inputCols=['object_sparse', 'action_sparse',
                                       'image_path_sparse', 'parent_image_path_sparse'], 
                            outputCol="final_vec")
df_test_final = assembler.transform(df_test)

In [42]:
df_test_final.limit(1).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,timestamp,malicious,object_sparse,action_sparse,image_path_sparse,parent_image_path_sparse,final_vec
0,1,2019-09-23 18:44:59.171,1,"(1.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."


In [70]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

# assume 'binary_column' is a DataFrame with a column named 'binary_int_array' containing dense binary int arrays

# define a user-defined function to convert binary int array to string array
def binary_to_string_array(binary_int_array):
    string_array = []
    for i in binary_int_array:
        string_array.append(str(int(i)))
    return ''.join(string_array)

# register the user-defined function as a UDF
binary_to_string_array_udf = udf(binary_to_string_array, StringType())

# use the UDF to create a new DataFrame column with string arrays
df_test_final = df_test_final.withColumn("string_array", binary_to_string_array_udf("final_vec"))

In [71]:
df_test_final.limit(1).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,timestamp,malicious,object_sparse,action_sparse,image_path_sparse,parent_image_path_sparse,final_vec,string_array,vec2str,event_index
0,1,2019-09-23 18:44:59.171,1,"(1.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",10000100000010000000000100,1.00.00.00.00.01.00.00.00.00.00.00.01.00.00.00...,3.0


In [62]:
df_test_final = df_test_final.withColumn('vec2str',concat_ws('',col('string_array')))
df_test_final.limit(1).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,timestamp,malicious,object_sparse,action_sparse,image_path_sparse,parent_image_path_sparse,final_vec,string_array,vec2str,event_index
0,1,2019-09-23 18:44:59.171,1,"(1.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",1.00.00.00.00.01.00.00.00.00.00.00.01.00.00.00...,3.0


In [75]:
#now we string index and we get the hell out of here

#indexer = StringIndexer(inputCol='string_array', outputCol='event_index_2')
#indexer_fitted = indexer.fit(df_test_final)
#df_test_final = indexer_fitted.transform(df_test_final)
def int_cast(num):
    return int(num)
int_cast_udf = udf(int_cast, IntegerType())
df_test_final = df_test_final.withColumn("ind_event", int_cast_udf("event_index_2"))

In [76]:
df_test_final.limit(1).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,timestamp,malicious,object_sparse,action_sparse,image_path_sparse,parent_image_path_sparse,final_vec,string_array,vec2str,event_index,event_index_2,ind_event
0,1,2019-09-23 18:44:59.171,1,"(1.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",10000100000010000000000100,1.00.00.00.00.01.00.00.00.00.00.00.01.00.00.00...,3.0,3.0,3


In [81]:
#what is the range of vlaues in ind_event
print(str(max(df_test_final.ind_event)))
df_test_final.agg(max(col('ind_event'))).show()



Column<'max(ind_event)'>


+--------------+
|max(ind_event)|
+--------------+
|            17|
+--------------+



+--------------+
|max(ind_event)|
+--------------+
|            17|
+--------------+



In [82]:
df_test_final.agg(min(col('ind_event'))).show()

+--------------+
|min(ind_event)|
+--------------+
|             0|
+--------------+



In [88]:
%run trace_encode.ipynb
df_test, dict_test = trace_encode(connected_paths,list_cols)

transposed explode: 0.22954916954040527
one-hot time: 5.508583068847656


indexing time: 26.78142738342285
total elapsed time: 26.79460382461548


In [87]:
df_test.limit(15).toPandas()

,Trace,malicious,vec2string,event_index
0,1,1,10000100000010000000000100,3
1,1,1,10001000000010000000001000,0
2,1,1,10001000000010000000001000,0
3,1,1,10001000000010000000001000,0
4,1,1,10001000000010000000001000,0
5,1,1,01000010000010000000000001,2
6,2,1,10001000000010000000001000,0
7,2,1,10001000000010000000001000,0
8,2,1,10001000000010000000001000,0
9,2,1,10001000000010000000001000,0


In [1]:
#new test df
%run ./etl_trusted_features.ipynb

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/19 14:17:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/19 14:17:38 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
df = readFirstEvents()

 2:18PM UTC on Mar 19, 2023 --- read time: 6.950451135635376 seconds ---


In [4]:
df_ben = df.where(col('malicious') == 0)
print(df_ben.count())
df_mal = df.where(col('malicious') == 1)
print("mal count: "+str(df_mal.count()))

981985
mal count: 3642


In [112]:
df_benign = df_benign.limit(30000)

In [113]:
from graphframes import GraphFrame

#create trace matrix from malicious events for speed. 
# Create distinct vertices with source as actorid, destination as objectid for malicious
src_vertices = df_benign.selectExpr('actorID as id').distinct()
dst_vertices = df_benign.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_benign.selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                      'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d); (d)-[e4]->(e); (e)-[e5]->(f); (f)-[e6]->(g)")

#create paths and count
# filter paths to only those where all edges are connected
connected_paths = motifs6.filter("e1.dst = e2.src and e2.dst = e3.src and e3.dst = e4.src and e4.dst = e5.src and e5.dst = e6.src").cache()
print("event traces: "+str(connected_paths.count()))

event traces: 16244380


In [6]:
#read in connected paths from parquet because of graphframes issues
connected_paths = spark.read.parquet(f"{s3_url_trusted}/prod/graph/test").cache()
print(connected_paths.count())
connected_paths.limit(2).toPandas()

21135


,a,e1,b,e2,c,e3,d,e4,e,e5,f,e6,g
0,"(2d02ab42-4400-4049-baad-6dfae19c3c4c,)","(2d02ab42-4400-4049-baad-6dfae19c3c4c, 2d02ab4...","(2d02ab42-4400-4049-baad-6dfae19c3c4c,)","(2d02ab42-4400-4049-baad-6dfae19c3c4c, 2d02ab4...","(2d02ab42-4400-4049-baad-6dfae19c3c4c,)","(2d02ab42-4400-4049-baad-6dfae19c3c4c, 2d02ab4...","(2d02ab42-4400-4049-baad-6dfae19c3c4c,)","(2d02ab42-4400-4049-baad-6dfae19c3c4c, 2d02ab4...","(2d02ab42-4400-4049-baad-6dfae19c3c4c,)","(2d02ab42-4400-4049-baad-6dfae19c3c4c, 2d02ab4...","(2d02ab42-4400-4049-baad-6dfae19c3c4c,)","(2d02ab42-4400-4049-baad-6dfae19c3c4c, c3d7e86...","(c3d7e86e-fd61-4980-a2a4-8ddb7d5ce080,)"
1,"(2d02ab42-4400-4049-baad-6dfae19c3c4c,)","(2d02ab42-4400-4049-baad-6dfae19c3c4c, 2d02ab4...","(2d02ab42-4400-4049-baad-6dfae19c3c4c,)","(2d02ab42-4400-4049-baad-6dfae19c3c4c, 2d02ab4...","(2d02ab42-4400-4049-baad-6dfae19c3c4c,)","(2d02ab42-4400-4049-baad-6dfae19c3c4c, 2d02ab4...","(2d02ab42-4400-4049-baad-6dfae19c3c4c,)","(2d02ab42-4400-4049-baad-6dfae19c3c4c, 2d02ab4...","(2d02ab42-4400-4049-baad-6dfae19c3c4c,)","(2d02ab42-4400-4049-baad-6dfae19c3c4c, 2d02ab4...","(2d02ab42-4400-4049-baad-6dfae19c3c4c,)","(2d02ab42-4400-4049-baad-6dfae19c3c4c, 8ae4dd9...","(8ae4dd98-1e4c-4269-bba8-2823e84b439b,)"


In [115]:
#generate a test data frame to re-transpose
%run trace_encode.ipynb
df_test, dict_test = trace_encode(connected_paths,list_cols)

transposed explode: 0.28693699836730957


one-hot time: 26.595794677734375


indexing time: 220.06580138206482
total elapsed time: 220.12458777427673


In [21]:
#generate a test data frame to re-transpose
%run trace_encode.ipynb
list_cols = ['object','action',
             'image_path','parent_image_path']
df_test, dict_test = trace_encode(connected_paths,list_cols)

transposed explode: 0.12774348258972168
one-hot time: 1.9131495952606201


indexing time: 22.757339000701904
total elapsed time: 22.762881755828857


In [11]:
#run sample code from chat GPT
# Create a sample dataframe
df = spark.createDataFrame([(1, 'A', 10, 100),
                            (1, 'B', 20, 200),
                            (2, 'A', 30, 300),
                            (2, 'B', 40, 400)],
                           ['id', 'series', 'value1', 'value2'])

# Pivot the dataframe
pivoted_df = df.groupBy('id').pivot('series').agg(first('value1'), first('value2'))

# Show the pivoted dataframe
pivoted_df.show()

+---+---------------+---------------+---------------+---------------+
| id|A_first(value1)|A_first(value2)|B_first(value1)|B_first(value2)|
+---+---------------+---------------+---------------+---------------+
|  1|             10|            100|             20|            200|
|  2|             30|            300|             40|            400|
+---+---------------+---------------+---------------+---------------+



In [25]:
df_test.limit(15).toPandas()

,Trace,event,malicious,vec2string,event_index
0,1,e1,1,10001000000010000000001000,0
1,1,e2,1,10001000000010000000001000,0
2,1,e3,1,10001000000010000000001000,0
3,1,e4,1,10001000000010000000001000,0
4,1,e5,1,10001000000010000000001000,0
5,1,e6,1,01000010000010000000000001,2
6,2,e1,1,10001000000010000000001000,0
7,2,e2,1,10001000000010000000001000,0
8,2,e3,1,10001000000010000000001000,0
9,2,e4,1,10001000000010000000001000,0


In [45]:
#Todo set max pivot values
#spark.sql.pivotMaxValues = df_test.select('Trace').distinct().count()

pivot_test = df_test.groupBy('Trace').pivot('event').agg(first('malicious'),
                                        first('vec2string'),
                                        first('event_index'))
pivot_test.limit(3).toPandas()

,Trace,e1_first(malicious),e1_first(vec2string),e1_first(event_index),e2_first(malicious),e2_first(vec2string),e2_first(event_index),e3_first(malicious),e3_first(vec2string),e3_first(event_index),e4_first(malicious),e4_first(vec2string),e4_first(event_index),e5_first(malicious),e5_first(vec2string),e5_first(event_index),e6_first(malicious),e6_first(vec2string),e6_first(event_index)
0,1,1,10001000000010000000001000,0,1,10001000000010000000001000,0,1,10001000000010000000001000,0,1,10001000000010000000001000,0,1,10001000000010000000001000,0,1,01000010000010000000000001,2
1,2,1,10001000000010000000001000,0,1,10001000000010000000001000,0,1,10001000000010000000001000,0,1,10001000000010000000001000,0,1,10001000000010000000001000,0,1,01000010000010000000000001,2
2,3,1,10001000000010000000001000,0,1,10001000000010000000001000,0,1,10001000000010000000001000,0,1,10001000000010000000001000,0,1,10001000000010000000001000,0,1,01000010000010000000000001,2


In [46]:
pivot_test_vec = df_test.groupBy('Trace').pivot('event').agg(first('malicious'),
                                        first('vec2string'))
                                    
pivot_test_vec.limit(10).toPandas()

,Trace,e1_first(malicious),e1_first(vec2string),e2_first(malicious),e2_first(vec2string),e3_first(malicious),e3_first(vec2string),e4_first(malicious),e4_first(vec2string),e5_first(malicious),e5_first(vec2string),e6_first(malicious),e6_first(vec2string)
0,1,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,01000010000010000000000001
1,2,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,01000010000010000000000001
2,3,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,01000010000010000000000001
3,4,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,01000010000010000000000001
4,5,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10000100000000100000001000
5,6,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000
6,7,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,01000010000010000000000001
7,8,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,01000010000010000000000001
8,9,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,01000010000010000000000001
9,10,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,10001000000010000000001000,1,01000010000010000000000001


In [47]:
pivot_test_ind = df_test.groupBy('Trace').pivot('event').agg(first('malicious'),
                                        first('event_index'))
pivot_test_ind.count()
#now I just have to drop a rename the columns and I am good 


21135

In [51]:
#now add all the columns into a single array and drop everything else
ind_final = pivot_test_ind.select('Trace',col('e1_first(malicious)').alias('malicious'),
                                  array('e1_first(event_index)', 'e2_first(event_index)',
                           'e3_first(event_index)','e4_first(event_index)', 
                           'e5_first(event_index)', 'e6_first(event_index)').alias('event_sequence'))
ind_final.sort('Trace').limit(10).toPandas()

,Trace,malicious,event_sequence
0,1,1,"[0, 0, 0, 0, 0, 2]"
1,2,1,"[0, 0, 0, 0, 0, 2]"
2,3,1,"[0, 0, 0, 0, 0, 2]"
3,4,1,"[0, 0, 0, 0, 0, 2]"
4,5,1,"[0, 0, 0, 0, 0, 10]"
5,6,1,"[0, 0, 0, 0, 0, 0]"
6,7,1,"[0, 0, 0, 0, 0, 2]"
7,8,1,"[0, 0, 0, 0, 0, 2]"
8,9,1,"[0, 0, 0, 0, 0, 2]"
9,10,1,"[0, 0, 0, 0, 0, 2]"


In [49]:
vec_final = pivot_test_vec.select('Trace',col('e1_first(malicious)').alias('malicious'),
                                  array('e1_first(vec2string)', 'e2_first(vec2string)',
                           'e3_first(vec2string)','e4_first(vec2string)', 
                           'e5_first(vec2string)', 'e6_first(vec2string)').alias('event_sequence'))

vec_final.limit(10).toPandas()

,Trace,malicious,event_sequence
0,1,1,"[10001000000010000000001000, 10001000000010000..."
1,2,1,"[10001000000010000000001000, 10001000000010000..."
2,3,1,"[10001000000010000000001000, 10001000000010000..."
3,4,1,"[10001000000010000000001000, 10001000000010000..."
4,5,1,"[10001000000010000000001000, 10001000000010000..."
5,6,1,"[10001000000010000000001000, 10001000000010000..."
6,7,1,"[10001000000010000000001000, 10001000000010000..."
7,8,1,"[10001000000010000000001000, 10001000000010000..."
8,9,1,"[10001000000010000000001000, 10001000000010000..."
9,10,1,"[10001000000010000000001000, 10001000000010000..."


In [53]:
#now let's test it for index
%run trace_encode.ipynb
list_cols = ['object','action',
             'image_path','parent_image_path']
df_test, dict_test = trace_encode(connected_paths,list_cols)

transposed explode: 0.12270402908325195
one-hot time: 1.8612608909606934


indexing time: 22.65959358215332
total elapsed time: 22.86321783065796


In [59]:
print(df_test.count())
df_test.limit(10).toPandas()

21135


,Trace,malicious,event_sequence
0,4590,1,"[3, 0, 0, 0, 3, 6]"
1,5409,1,"[3, 0, 3, 0, 3, 2]"
2,11190,1,"[1, 0, 0, 1, 0, 0]"
3,11567,1,"[0, 0, 0, 1, 0, 0]"
4,7747,1,"[0, 0, 1, 0, 0, 0]"
5,8589935298,1,"[0, 0, 0, 0, 0, 2]"
6,8589935936,1,"[0, 0, 0, 0, 0, 2]"
7,8589936424,1,"[0, 0, 0, 0, 0, 2]"
8,1950,1,"[0, 0, 0, 0, 0, 0]"
9,2509,1,"[3, 0, 0, 0, 0, 2]"


In [54]:
#now test it for vec
%run trace_encode.ipynb
list_cols = ['object','action',
             'image_path','parent_image_path']
df_test_vec, dict_test = trace_encode(connected_paths,list_cols, output = 'vec')

transposed explode: 0.11651253700256348
one-hot time: 1.6763536930084229
total elapsed time: 1.8925683498382568


In [58]:
print(df_test_vec.count())
df_test_vec.limit(10).toPandas()

21135


,Trace,malicious,event_sequence
0,1,1,"[10001000000010000000001000, 10001000000010000..."
1,2,1,"[10001000000010000000001000, 10001000000010000..."
2,3,1,"[10001000000010000000001000, 10001000000010000..."
3,4,1,"[10001000000010000000001000, 10001000000010000..."
4,5,1,"[10001000000010000000001000, 10001000000010000..."
5,6,1,"[10001000000010000000001000, 10001000000010000..."
6,7,1,"[10001000000010000000001000, 10001000000010000..."
7,8,1,"[10001000000010000000001000, 10001000000010000..."
8,9,1,"[10001000000010000000001000, 10001000000010000..."
9,10,1,"[10001000000010000000001000, 10001000000010000..."


In [2]:
#experiment with binning first to make sure it's read in, and then to make sure we have the right amount 
#of bins
df = readFirstEvents()

 1:40AM UTC on Mar 24, 2023 --- read time: 7.3005523681640625 seconds ---


In [3]:
df_m_23 = df.where((col('malicious') == 1) & (col('event_day')==23))

In [4]:
df_m_23.count()

3624

In [12]:
%run trace_encode.ipynb

In [6]:
df_graph = create_graph(df_m_23)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


event traces: 1261


In [8]:
#add "trace index" to keep track of traces. important for transposing back
df_transp = df_graph.withColumn("Trace", (monotonically_increasing_id() + 1))
df_transp = df_transp.select("Trace", 
                             *[col for col in df_transp.columns if col != "Trace"])

#drop all vertices
df_transp = df_transp.drop('a','b','c','d','e','f','g')

#transpose rows 
stacked_df = df_transp.selectExpr(
    "Trace", 
    "posexplode(array(e1, e2, e3, e4, e5, e6)) as (pos, col)"
).select(
    "Trace", 
    expr('''CASE pos 
    WHEN 0 THEN 'e1' 
    WHEN 1 THEN 'e2'
    WHEN 2 THEN 'e3'
    WHEN 3 THEN 'e4'
    WHEN 4 THEN 'e5'
    ELSE 'e6' END''').alias("event"),
    "col"
).orderBy("Trace","event")

#explode columns
df_onehot = stacked_df.select(*stacked_df.columns, "col.*").drop('col')

In [11]:
df_ts = ts_diff(df_onehot)
df_ts.limit(24).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,image_path,parent_image_path,new_path,file_path,direction,logon_id,requesting_domain,requesting_user,malicious,timestamp_difference
0,1,e1,f5d1c003-1f65-4a8a-ba8c-413da99d70f2,a52f6519-6265-4437-9f6c-3402fb245cc7,2019-09-23 15:25:52.486,PROCESS,CREATE,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,None,None,None,None,1,NaN
1,1,e3,f5d1c003-1f65-4a8a-ba8c-413da99d70f2,a52f6519-6265-4437-9f6c-3402fb245cc7,2019-09-23 15:25:52.486,PROCESS,CREATE,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,None,None,None,None,1,0.000000e+00
2,1,e5,f5d1c003-1f65-4a8a-ba8c-413da99d70f2,a52f6519-6265-4437-9f6c-3402fb245cc7,2019-09-23 15:25:52.486,PROCESS,CREATE,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,None,None,None,None,1,0.000000e+00
3,1,e2,a52f6519-6265-4437-9f6c-3402fb245cc7,f5d1c003-1f65-4a8a-ba8c-413da99d70f2,2019-09-23 15:40:21.404,PROCESS,OPEN,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,None,None,None,None,1,8.689180e+05
4,1,e4,a52f6519-6265-4437-9f6c-3402fb245cc7,f5d1c003-1f65-4a8a-ba8c-413da99d70f2,2019-09-23 15:40:21.404,PROCESS,OPEN,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,None,None,None,None,1,0.000000e+00
5,1,e6,a52f6519-6265-4437-9f6c-3402fb245cc7,36d062d4-1f69-4dcc-9791-46b5c07dadef,2019-09-23 17:21:39.921,FILE,READ,SysClient0201.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,\Device\HarddiskVolume1\Windows\system32\Windo...,None,None,None,None,1,6.078517e+06
6,2,e1,4aec5900-4572-4d09-81e6-fbc227a7f58d,3eea9071-9ceb-468a-ad89-9e3890833e52,2019-09-23 17:34:17.240,PROCESS,CREATE,SysClient0660.systemia.com,None,None,powershell.exe,\Device\HarddiskVolume1\Windows\system32\wbem\...,None,None,None,None,None,None,1,NaN
7,2,e3,4aec5900-4572-4d09-81e6-fbc227a7f58d,3eea9071-9ceb-468a-ad89-9e3890833e52,2019-09-23 17:34:17.240,PROCESS,CREATE,SysClient0660.systemia.com,None,None,powershell.exe,\Device\HarddiskVolume1\Windows\system32\wbem\...,None,None,None,None,None,None,1,0.000000e+00
8,2,e5,4aec5900-4572-4d09-81e6-fbc227a7f58d,3eea9071-9ceb-468a-ad89-9e3890833e52,2019-09-23 17:34:17.240,PROCESS,CREATE,SysClient0660.systemia.com,None,None,powershell.exe,\Device\HarddiskVolume1\Windows\system32\wbem\...,None,None,None,None,None,None,1,0.000000e+00
9,2,e2,3eea9071-9ceb-468a-ad89-9e3890833e52,4aec5900-4572-4d09-81e6-fbc227a7f58d,2019-09-23 17:43:13.564,PROCESS,OPEN,SysClient0660.systemia.com,None,None,powershell.exe,powershell.exe,None,None,None,None,None,None,1,5.363240e+05


In [13]:
#last function to test
df_m = bin_it(df_ts)
df_m.limit(2).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,...,parent_image_path,new_path,file_path,direction,logon_id,requesting_domain,requesting_user,malicious,timestamp_difference,timestamp_bins
0,1,e1,f5d1c003-1f65-4a8a-ba8c-413da99d70f2,a52f6519-6265-4437-9f6c-3402fb245cc7,2019-09-23 15:25:52.486,PROCESS,CREATE,SysClient0201.systemia.com,None,None,...,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,None,None,None,None,1,NaN,NaN
1,1,e3,f5d1c003-1f65-4a8a-ba8c-413da99d70f2,a52f6519-6265-4437-9f6c-3402fb245cc7,2019-09-23 15:25:52.486,PROCESS,CREATE,SysClient0201.systemia.com,None,None,...,\Device\HarddiskVolume1\Windows\System32\Windo...,None,None,None,None,None,None,1,0.0,1.0


In [7]:
#testing to see if motif2 produces more graphs than 6 and here it did.
# Create distinct vertices with source as actorid, destination as objectid
src_vertices = df_m_23.selectExpr('actorID as id').distinct()
dst_vertices = df_m_23.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_m_23.selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                      'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)

# find all paths of length 6 in the graph
# Handling null values in shorter length 
motifs2 = g.find("(a)-[e1]->(b); (b)-[e2]->(c)")

motifs2.count()

3608